# <center><font color='red'><b>JanataHack: Computer Vision Hackathon</b></font></center>

___

<center>15 Day Hackathon From Analytics Vidhya</center>

<center><img src="https://datahack-prod.s3.ap-south-1.amazonaws.com/__sized__/contest_cover/cover_FJJYomD-thumbnail-1200x1200-90.jpg"/>

## About the Hackathon

__Fatalities__ due to traffic delays of emergency vehicles such as __ambulance & fire brigade__ is a huge problem. In daily life, we often see that emergency vehicles face difficulty in passing through traffic. So differentiating a vehicle into an emergency and non emergency category can be an important component in __traffic monitoring__ as well as self drive car systems as reaching on time to their destination is critical for these services.

______

## Task 

In this problem, you will be working on __classifying vehicle images__ as either belonging to the ```emergency vehicle or non-emergency vehicle category```. 

- For the same, you are provided with the __train__ and the __test dataset__. Emergency vehicles usually includes ```police cars, ambulance and fire brigades```.

## Sample Images

<center><img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/08/Emgen.jpg"/></center>

## Data Description
- ```train.zip```: contains 2 csvs and 1 folder containing image data
- ```train.csv``` – [‘image_names’, ‘emergency_or_not’] contains the image name and correct class for 1646 (70%) train images.

- ```images``` – contains __2352__ images for both train and test sets

- ```test.csv```: [‘image_names’] contains just the image names for the 706 (30%) test images

- ```sample_submission.csv```: [‘image_names’,’emergency_or_not­’] contains the exact format for a valid submission (1 - For Emergency Vehicle, 0 - For Non Emergency Vehicle)

## Evaluation Metric
- The evaluation metric for this competition is ```Accuracy```.

### Importing the necessary libraries

In [ ]:
# To print multiple output in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# from google.colab.patches import cv2_imshow

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#from albumentations import *
import cv2
import copy
import os
print(os.listdir("../input"))

#!pip install pretrainedmodels
from tqdm import tqdm_notebook as tqdm
from torchvision.models import *
#import pretrainedmodels

from fastai.vision import *
from fastai.vision.models import *
from fastai.vision.learner import model_meta
from fastai.callbacks import * 

#from utils import *
import sys

# Any results you write to the current directory are saved as output.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from sklearn.metrics import f1_score
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import glob
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch
from fastai.callbacks.hooks import *

In [ ]:
## set the data folder
data_folder = Path("../input/hp-2020/jh_2020")

In [ ]:
print(os.listdir(data_folder))

In [ ]:
# ../input/hp-2020/jh_2020/images

In [ ]:
print(os.listdir('../input/hp-2020/jh_2020/images'))

In [ ]:
data_path = "../input/hp-2020/jh_2020/images/"
path = os.path.join(data_path , "*jpg")


In [ ]:
print(os.listdir(data_path))

In [ ]:
files = glob.glob(path)
data=[]
for file in files:
    image = cv2.imread(file)
    data.append(image)

In [ ]:
# data[1]

In [ ]:
## read the csv data files
train_df = pd.read_csv('../input/hp-2020/jh_2020/train.csv')
test_df = pd.read_csv('../input/hp-2020/jh_2020/test.csv')
submit = pd.read_csv('../input/hp-2020/jh_2020/sample_submission.csv')

In [ ]:
print(test_df.shape)
print(train_df.shape)

In [ ]:
train_images = data[:1646]
test_images= data[1646:]

In [ ]:
train_df.head(2)

In [ ]:
train_df['emergency_or_not'].value_counts()

- Can clearly see there is an imbalance in the both clases

### In this Section of code we gonna duplicate a set of 150 image of class='1' 

- Choose a random sample of 150 Images

In [ ]:
dupli=train_df[train_df['emergency_or_not']==1].sample(150)

In [ ]:
dupli.shape
dupli.head(4)

### Number of images before and shape of training data before.

In [ ]:
# path="/content/images"
print(train_df.shape)


### Creating duplicate images

In [ ]:
# for i in dupli['image_names']:    
#     new_image='copy'+i
#     original_img = cv2.imread("/content/images/"+i)
# #     clone_img = copy.copy(original_img)
# #     data.append(clone_img)
#     cv2.imwrite(data_path + new_image,original_img)

#     d=[{'image_names':new_image,'emergency_or_not':1}]

#     train_df=train_df.append(d,ignore_index=True,sort=False)

### Number of images before and shape of training data after.

In [ ]:
# print(train_df.shape)
# len(listdir(path))

- So here you can check the images in training data increased to 1796 and number of images increased to 2502

In [ ]:
train_df['emergency_or_not'].value_counts()

## Data Transformation

In [ ]:

##transformations to be done to images
tfms = get_transforms(do_flip=False,flip_vert=False ,max_rotate=10.0, max_zoom=1.22, max_lighting=0.22, max_warp=0.4, p_affine=0.75,
                      p_lighting=0.75)
#, xtra_tfms=zoom_crop(scale=(0.9,1.8), do_rand=True, p=0.8))


#Apply new transformations

# tfms = get_transforms(do_flip=False,flip_vert=False ,max_rotate=10.0, max_zoom=1.22, max_lighting=0.22, max_warp=0.4, p_affine=0.75,
#                       p_lighting=0.75, xtra_tfms=zoom_crop(scale=(0.9,1.8), do_rand=True, p=0.8))

## create databunch of test set to be passed
test_img = ImageList.from_df(test_df, path=data_folder, folder='images')

In [ ]:
np.random.seed(45)
## create source of train image databunch
src = (ImageList.from_df(train_df, path=data_folder, folder='images')
       .split_by_rand_pct(0.2)
       #.split_none()
       .label_from_df()
       .add_test(test_img))

- Previously we did the whole training with image size=400 and all the time validation_accuracy revolve around 97% and the test accuracy is around 96%
- This time try with 300 image size(because in my previous training i achieve the best score of 97% with size=300)

- Beware that we have to adjust batchsize accordingly to run out of memory.

In [ ]:
data = (src.transform(tfms, size=300,padding_mode='reflection',resize_method=ResizeMethod.SQUISH)
        .databunch(path='.', bs=16, device= torch.device('cuda:0')).normalize(imagenet_stats))

# data = (src.transform(tfms, size=350,padding_mode='reflection',resize_method=ResizeMethod.SQUISH)
#         .databunch(path='.', bs=16).normalize(imagenet_stats))

In [ ]:
print(data.classes)

In [ ]:
data.show_batch(rows=3, figsize=(7,7))

## Prepare the cnn_learner

In [ ]:
#lets create learner. tried with resnet152, densenet201, resnet101
# learn = cnn_learner(data=data, base_arch=models.densenet201, metrics=[FBeta(beta=1, average='macro'),error_rate],
#                     callback_fns=ShowGraph)



learn = cnn_learner(data=data, base_arch=models.densenet201, metrics=[FBeta(beta=1, average='macro'), accuracy],
                    callback_fns=ShowGraph).mixup()

In [ ]:
# learn.summary()

In [ ]:

#lets find the correct learning rate to be used from lr finder
learn.lr_find()
learn.recorder.plot(suggestion=True)

> ## New methodology
> 

In [ ]:
#lets start with steepset slope point. adding wd (weight decay) not to overfit as we are running 10 epochs 
lr = 1e-04
#learn.fit_one_cycle(10, slice(lr))
# learn.fit_one_cycle(10, max_lr=slice(lr), wd=0.2)

# learn.fit_one_cycle(5, max_lr=slice(3e-5,3e-2))         # previous


# learn.fit_one_cycle(5, max_lr=slice(3e-5,1e-3),wd=0.2)   # with this we got better accuracy 

# for first layer don't use slice as it used to train other layers.

learn.fit_one_cycle(10, slice(lr),wd=0.1)   

In [ ]:
learn.fit_one_cycle(10, slice(lr),wd=0.1) 

In [ ]:
# save the model

learn.save('stage1_model')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(dpi=120)

Now Unfreeze entire model.This Sets every layer group to trainable (i.e. requires_grad=True)

In [ ]:
#lets plot the lr finder record
learn.unfreeze()
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
# train for  more cycles after unfreezing
# learn.fit_one_cycle(5,slice(1e-05),wd=0.15)
learn.fit_one_cycle(10, slice(1e-03, lr/10),wd=0.15)

In [ ]:
# save the model

learn.save('stage2_model')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(dpi=120)

- Didn't see much improvement.


lets freeze the all layers except last 3 as these are initial layers for finding recurring pattern/ shapes/corners etc. (not exactly helpful in finding ships). so its better not to change stats of those layers

In [ ]:
learn.freeze_to(-3)

In [ ]:
## finding the LR
learn.lr_find()
learn.recorder.plot(suggestion=True)

- Train for few more cycles (we will be setting two LRs in below trainings: first one to train the initial layers and second to train last layers ).
- As initial layers' stats are imagenet stats which are helpful in finding patterns (discussed above) not the exact ships, so we will be training those layers with very low learning rates (to not to greatly change those initial layer parameters)

In [ ]:
# 96.52% accuracy achieved
learn.fit_one_cycle(6, slice(1e-02, lr/10),wd=0.15)

In [ ]:
# save the model

learn.save('stage3_model')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(dpi=120)

In [ ]:
## freezing initial all layers except last 2 layers

# 96.18%
learn.freeze_to(-2)

In [ ]:
## finding the LR
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
## training for few cylcles more
learn.fit_one_cycle(5, slice(5e-04, lr/40),wd=0.15)

In [ ]:
## training for few cylcles more
learn.fit_one_cycle(5, slice(5e-04, lr/100),wd=0.15)

In [ ]:
# save the model

learn.save('stage4_model_new')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(dpi=120)

In [ ]:
learn.freeze_to(-1)

In [ ]:
## finding the LR
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
## training even more
learn.fit_one_cycle(5, slice(1e-05, lr/30),wd=0.05)

In [ ]:
learn.fit_one_cycle(6, slice(5e-06, lr/100))

### Accuracy check

In [ ]:

##learn.TTA improves score further. lets see for the validation set
pred_val,y = learn.TTA(ds_type=DatasetType.Valid)
from sklearn.metrics import f1_score, accuracy_score
valid_preds = [np.argmax(pred_val[i]) for i in range(len(pred_val))]
valid_preds = np.array(valid_preds)
y = np.array(y)
accuracy_score(valid_preds,y),f1_score(valid_preds,y, average='micro')


In [ ]:
# y

In [ ]:
# preds,y = learn.TTA(ds_type=DatasetType.Test)
preds,_ = learn.get_preds(ds_type = DatasetType.Test)
labelled_preds = [np.argmax(preds[i]) for i in range(len(preds))]

labelled_preds = np.array(labelled_preds)

In [ ]:
# test_df.shape

In [ ]:
len(labelled_preds)

## FINAL PREDICTION FOR SUBMISSION

In [ ]:
#create submission file
df = pd.DataFrame({'image_names':test_df['image_names'], 'emergency_or_not':labelled_preds}, columns=['image_names', 'emergency_or_not'])
df.to_csv('submission_model_new1.csv', index=False)

In [ ]:
d=pd.read_csv('submission1.csv')
d.head()

- At first glance fastai seems to be bit complicated but as you learn it became easy to work with it.

### Suggestions: 

- Maintain a proper logsheet for you models,parameters and pretrained models,submission(It will help you to track you progress).
- Try as many methods as you can.
- While training a model you need fast processing so its better to use kaggle GPU ( Can also use Google collab GPU but over there processing is very slow, for an estimate 

    - Kaggle Training time`~10mins
    - Google Training time ~1:30hr

## Logsheet Sample

- You can design something like this for your model to track the training progress. I know it is a bit tedious do track all progress side by side, but believe me, if you want to feature among top best then track down all your progress.

<img src="https://raw.githubusercontent.com/AIVenture0/JanataHack-Computer-Vision-Hackathon-21st-Rank-/master/logsheet.jpg"/>

#### For this competition i also used keras for with MobileNet pretrained model. 

[Check this out](https://github.com/AIVenture0/JanataHack-Computer-Vision-Hackathon-21st-Rank-)

1. ### - Do upvote
1. ### - Share
1. ### - For queries use ```comment section```